# Data labeling with `edsl`
This notebook shows how to use `edsl` to automate data labeling and content analysis. In a series of steps we design and conduct an analysis of customer service tickets for a transportation app.

## Technical setup
This section contains code for importing the basic `edsl` tools that we will use to design and conduct our content analysis. The tools allow us to define the ticket content review as a survey of questions that we deliver to a specified lanaguage model, with carefully constructed directions about the task to be done and the desired format of results.

In [1]:
# ! pip install edsl

In [2]:
from edsl.questions import QuestionMultipleChoice, QuestionCheckBox, QuestionFreeText, QuestionList, QuestionYesNo, QuestionLinearScale
from edsl import Survey, Scenario, Model

We can see the available models and choose one or more of them:

In [3]:
Model.available()

['claude-3-haiku-20240307',
 'claude-3-opus-20240229',
 'claude-3-sonnet-20240229',
 'dbrx-instruct',
 'gemini_pro',
 'gpt-3.5-turbo',
 'gpt-4-1106-preview',
 'llama-2-13b-chat-hf',
 'llama-2-70b-chat-hf',
 'mixtral-8x7B-instruct-v0.1']

In [4]:
model = Model('gpt-4-1106-preview')

## Assessment design
We start by identifying some content for review. Content can be created or imported from existing files. For purposes of this demo, we'll import a set of hypothetical customer service ticket comments for a transportation app:

In [5]:
tickets = [
    "I just realized I left my phone in the car on my last ride. Can you help me get it back?",
    "I'm unhappy with my recent experience. The driver was very rude and unprofessional.",
    "I was charged more than the estimated fare for my trip yesterday. Can you explain why?",
    "The car seat provided was not properly installed, and I felt my child was at risk. Please ensure driver training.",
    "My driver took a longer route than necessary, resulting in a higher fare. I request a fare adjustment.",
    "I had a great experience with my driver today! Very friendly and efficient service.",
    "I'm concerned about the vehicle's cleanliness. It was not up to the standard I expect.",
    "The app keeps crashing every time I try to book a ride. Please fix this issue.",
    "My driver was exceptional - safe driving, polite, and the car was spotless. Kudos!",
    "I felt unsafe during my ride due to the driver's erratic behavior. This needs to be addressed immediately.",
    "The driver refused to follow my preferred route, which is shorter. I'm not satisfied with the service.",
    "Impressed with the quick response to my ride request and the driver's professionalism.",
    "I was charged for a ride I never took. Please refund me as soon as possible.",
    "The promo code I tried to use didn't work. Can you assist with this?",
    "There was a suspicious smell in the car, and I'm worried about hygiene standards.",
    "My driver was very considerate, especially helping me with my luggage. Appreciate the great service!",
    "The app's GPS seems inaccurate. It directed the driver to the wrong pick-up location.",
    "I want to compliment my driver's excellent navigation and time management during rush hour.",
    "The vehicle didn't match the description in the app. It was confusing and concerning.",
    "I faced an issue with payment processing after my last ride. Can you look into this?",
]

Next we identify some dimensions for our ticket review, constructed as a set of questions that we'll administer to an AI with the ticket contents. The questions could include the following:
<i>
    Check all of the issues mentioned in this ticket: safety, cleanliness, driver performance, etc.
    What is the primary issue in this ticket?
    If the primary issue is safety, was there an accident where someone was hurt?
    What is the sentiment of this ticket?
    Does the customer ask for a refund?
    On a scale from 1 (lowest) to 5 (highest), what is the priority level of this ticket?
</i>

The code that we use specifies the questions, question types and answer options, and allows us to select a particular language model. The questions are written in `edsl` format so that they can be sent to an AI and edited or replaced with questions of your choice.

In [6]:
question_issues = QuestionCheckBox(
    question_name="issues",
    question_text="Check all of the issues mentioned in this ticket: {{ticket}}",
    question_options=[
        "safety", 
        "cleanliness", 
        "driver performance", 
        "GPS/route", 
        "lost item", 
        "other"]
)

In [7]:
question_primary_issue = QuestionFreeText(
    question_name="primary_issue",
    question_text="What is the primary issue in this ticket? Ticket: {{ticket}}",
    allow_nonresponse=False,
)

In [8]:
question_accident = QuestionMultipleChoice(
    question_name="accident",
    question_text="If the primary issue in this ticket is safety, was there an accident where someone was hurt? Ticket: {{ticket}}",
    question_options=["Yes", "No", "Not applicable"]
)

In [9]:
question_sentiment = QuestionMultipleChoice(
    question_name="sentiment",
    question_text="What is the sentiment of this ticket? Ticket: {{ticket}}",
    question_options=[
        "Very positive",
        "Somewhat positive",
        "Neutral",
        "Somewhat negative",
        "Very negative"
    ]
)

In [10]:
question_refund = QuestionYesNo(
    question_name="refund",
    question_text="Does the customer ask for a refund in this ticket? Ticket: {{ticket}}"
)

In [11]:
question_priority = QuestionLinearScale(
    question_name="priority",
    question_text="On a scale from 0 (lowest) to 5 (highest), what is the priority level of this ticket? Ticket: {{ticket}}",
    question_options=[0,1,2,3,4,5]
)

In [12]:
survey = Survey(
    questions=[
        question_issues,
        question_primary_issue,
        question_accident,
        question_sentiment,
        question_refund,
        question_priority
    ]
)

We can also review our questions in a readable format, or export them as a survey to use with human respondents or at other survey platforms:

In [13]:
survey.print()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Questions                                                                                                       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃ Question Name ┃ Question Type ┃ Question Text                        ┃ Options                              ┃ │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│ │ issues        │ checkbox      │ Check all of the issues mentioned in │ safety, cleanliness, driver          │ │
│ │               │               │ this ticket: {{ticket}}              │ performance, GPS/route, lost item,   │ │
│ │               │               │                                      │ other                                │ │
│ └───────────────┴───────────────┴──────────────────────────────────────┴──────────────────────────────────────┘ │
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓      │
│ ┃ Question Name ┃ Question Type ┃ Question Text                                                ┃ Options ┃      │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩      │
│ │ primary_issue │ free_text     │ What is the primary issue in this ticket? Ticket: {{ticket}} │ None    │      │
│ └───────────────┴───────────────┴──────────────────────────────────────────────────────────────┴─────────┘      │
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃ Question Name ┃ Question Type   ┃ Question Text                                   ┃ Options                 ┃ │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│ │ accident      │ multiple_choice │ If the primary issue in this ticket is safety,  │ Yes, No, Not applicable │ │
│ │               │                 │ was there an accident where someone was hurt?   │                         │ │
│ │               │                 │ Ticket: {{ticket}}                              │                         │ │
│ └───────────────┴─────────────────┴─────────────────────────────────────────────────┴─────────────────────────┘ │
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃ Question Name ┃ Question Type   ┃ Question Text                       ┃ Options                             ┃ │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│ │ sentiment     │ multiple_choice │ What is the sentiment of this       │ Very positive, Somewhat positive,   │ │
│ │               │                 │ ticket? Ticket: {{ticket}}          │ Neutral, Somewhat negative, Very    │ │
│ │               │                 │                                     │ negative                            │ │
│ └───────────────┴─────────────────┴─────────────────────────────────────┴─────────────────────────────────────┘ │
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓ │
│ ┃ Question Name ┃ Question Type ┃ Question Text                                                     ┃ Options ┃ │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩ │
│ │ refund        │ yes_no        │ Does the customer ask for a refund in this ticket? Ticket:        │ Yes, No │ │
│ │               │               │ {{ticket}}                                                        │         │ │
│ └───────────────┴───────────────┴─────────────────────

In [14]:
survey

Survey(questions=[QuestionCheckBox(question_name = 'issues', question_text = 'Check all of the issues mentioned in this ticket: {{ticket}}', min_selections = None, max_selections = None, question_options = ['safety', 'cleanliness', 'driver performance', 'GPS/route', 'lost item', 'other'], short_names_dict = {}), QuestionFreeText(question_name = 'primary_issue', question_text = 'What is the primary issue in this ticket? Ticket: {{ticket}}', allow_nonresponse = False), QuestionMultipleChoice(question_name = 'accident', question_text = 'If the primary issue in this ticket is safety, was there an accident where someone was hurt? Ticket: {{ticket}}', question_options = ['Yes', 'No', 'Not applicable'], short_names_dict = {}), QuestionMultipleChoice(question_name = 'sentiment', question_text = 'What is the sentiment of this ticket? Ticket: {{ticket}}', question_options = ['Very positive', 'Somewhat positive', 'Neutral', 'Somewhat negative', 'Very negative'], short_names_dict = {}), QuestionYe

## Analysis
With our content and questions we're now ready to package our survey and deliver it to an AI. We do this by inserting the content of each ticket into each question as an independent "scenario" for review.

In [15]:
scenarios = [Scenario({'ticket':t}) for t in tickets] 

In [16]:
results = survey.by(scenarios).by(model).run()

In [17]:
results.columns

['agent.agent_name',
 'answer.accident',
 'answer.accident_comment',
 'answer.issues',
 'answer.issues_comment',
 'answer.primary_issue',
 'answer.priority',
 'answer.priority_comment',
 'answer.refund',
 'answer.refund_comment',
 'answer.sentiment',
 'answer.sentiment_comment',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.accident_system_prompt',
 'prompt.accident_user_prompt',
 'prompt.issues_system_prompt',
 'prompt.issues_user_prompt',
 'prompt.primary_issue_system_prompt',
 'prompt.primary_issue_user_prompt',
 'prompt.priority_system_prompt',
 'prompt.priority_user_prompt',
 'prompt.refund_system_prompt',
 'prompt.refund_user_prompt',
 'prompt.sentiment_system_prompt',
 'prompt.sentiment_user_prompt',
 'raw_model_response.accident_raw_model_response',
 'raw_model_response.issues_raw_model_response',
 'raw_model_response.pri

In [18]:
results.select('ticket','issues','primary_issue','accident','sentiment','refund','priority').print()

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┓
┃ scenario         ┃ answer           ┃ answer          ┃ answer         ┃ answer           ┃ answer  ┃ answer    ┃
┃ .ticket          ┃ .issues          ┃ .primary_issue  ┃ .accident      ┃ .sentiment       ┃ .refund ┃ .priority ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━┩
│ Impressed with   │ ['driver         │ There is no     │ Not applicable │ Very positive    │ No      │ 0         │
│ the quick        │ performance']    │ issue reported  │                │                  │         │           │
│ response to my   │                  │ in this ticket. │                │                  │         │           │
│ ride request and │                  │ The customer is │                │                  │         │           │
│ the driver's     │                  │ expressing      │                │                  │         │           │
│ professionalism. │                  │ satisfaction    │                │                  │         │           │
│                  │                  │ with the quick  │                │                  │         │           │
│                  │                  │ response to     │                │                  │         │           │
│                  │                  │ their ride      │                │                  │         │           │
│                  │                  │ request and the │                │                  │         │           │
│                  │                  │ driver's        │                │                  │         │           │
│                  │                  │ professionalis… │                │                  │         │           │
├──────────────────┼──────────────────┼─────────────────┼────────────────┼──────────────────┼─────────┼───────────┤
│ I want to        │ ['driver         │ The primary     │ No             │ Very positive    │ No      │ 0         │
│ compliment my    │ performance',    │ issue in this   │                │                  │         │           │
│ driver's         │ 'GPS/route']     │ ticket is not a │                │                  │         │           │
│ excellent        │                  │ problem but a   │                │                  │         │           │
│ navigation and   │                  │ positive        │                │                  │         │           │
│ time management  │                  │ feedback        │                │                  │         │           │
│ during rush      │                  │ regarding the   │                │                  │         │           │
│ hour.            │                  │ driver's        │                │                  │         │           │
│                  │                  │ excellent       │                │                  │         │           │
│                  │                  │ navigation and  │                │                  │         │           │
│                  │                  │ time management │                │                  │         │           │
│                  │                  │ skills during   │                │                  │         │           │
│                  │                  │ rush hour.      │                │                  │         │           │
├──────────────────┼──────────────────┼─────────────────┼────────────────┼──────────────────┼─────────┼───────────┤
│ I faced an issue │ ['other']        │ The primary     │ Not applicable │ Somewhat         │ No      │ 4         │
│ with payment     │                  │ issue in the    │                │ negative         │         │           │
│ processing after │                  │ ticket is a     │                │                  │         │           │
│ my last ride.    │                  │ problem with    

We can apply some lables to our table:

In [19]:
(results
 .select('ticket','issues','primary_issue','accident','sentiment','refund','priority')
 .print(pretty_labels={'scenario.ticket':'Ticket','answer.issues':'Issues','answer.primary_issue':'Primary issue','answer.accident':'Accident','answer.sentiment':'Sentiment','answer.refund':'Refund request','answer.priority':'Priority'})
)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Ticket         ┃ Issues         ┃ Primary issue   ┃ Accident       ┃ Sentiment      ┃ Refund request ┃ Priority ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ Impressed with │ ['driver       │ There is no     │ Not applicable │ Very positive  │ No             │ 0        │
│ the quick      │ performance']  │ issue reported  │                │                │                │          │
│ response to my │                │ in this ticket. │                │                │                │          │
│ ride request   │                │ The customer is │                │                │                │          │
│ and the        │                │ expressing      │                │                │                │          │
│ driver's       │                │ satisfaction    │                │                │                │          │
│ professionali… │                │ with the quick  │                │                │                │          │
│                │                │ response to     │                │                │                │          │
│                │                │ their ride      │                │                │                │          │
│                │                │ request and the │                │                │                │          │
│                │                │ driver's        │                │                │                │          │
│                │                │ professionalis… │                │                │                │          │
├────────────────┼────────────────┼─────────────────┼────────────────┼────────────────┼────────────────┼──────────┤
│ I want to      │ ['driver       │ The primary     │ No             │ Very positive  │ No             │ 0        │
│ compliment my  │ performance',  │ issue in this   │                │                │                │          │
│ driver's       │ 'GPS/route']   │ ticket is not a │                │                │                │          │
│ excellent      │                │ problem but a   │                │                │                │          │
│ navigation and │                │ positive        │                │                │                │          │
│ time           │                │ feedback        │                │                │                │          │
│ management     │                │ regarding the   │                │                │                │          │
│ during rush    │                │ driver's        │                │                │                │          │
│ hour.          │                │ excellent       │                │                │                │          │
│                │                │ navigation and  │                │                │                │          │
│                │                │ time management │                │                │                │          │
│                │                │ skills during   │                │                │                │          │
│                │                │ rush hour.      │                │                │                │          │
├────────────────┼────────────────┼─────────────────┼────────────────┼────────────────┼────────────────┼──────────┤
│ I faced an     │ ['other']      │ The primary     │ Not applicable │ Somewhat       │ No             │ 4        │
│ issue with     │                │ issue in the    │                │ negative       │                │          │
│ payment        │                │ ticket is a     │                │                │                │          │
│ processing     │                │ problem with    │                │                │                │          │
│ after my last  │                │ payment         │   

We can explore some built-in visualization methods:

In [20]:
results.bar_chart("accident")

In [21]:
results.bar_chart("sentiment")

In [22]:
results.bar_chart("refund")

In [23]:
results.bar_chart("priority")

In [24]:
results.tally("sentiment")

Very negative,4
Somewhat negative,11
Neutral,0
Somewhat positive,0
Very positive,5


In [25]:
results.tally("priority")

5,2
4,7
3,5
2,1
1,0
0,5


In [26]:
results.word_cloud_plot("issues")

---
<p style="font-size: 14px;">Copyright © 2024 Expected Parrot, Inc. All rights reserved.   <a href="www.expectedparrot.com" style="color:#130061">www.expectedparrot.com</a></p>